### CNN - Training, using MobileNetV2
 

In [ ]:
import os
import cv2
import numpy as np
import requests
from PIL import Image
import io
import time
from datetime import datetime
from matplotlib import pyplot as plt
from IPython.display import display, clear_output

# ==== USER SETTINGS ====
jetbot_ip = "194.47.156.140"
color_name = "yellow"         # Set to your color: "blue", "red", etc
desired_images = 100        # How many labeled frames to save
fps = 5                     # For Jetbot, 3-10 is practical for preview

# ==== HSV RANGES ====
color_hsv_ranges = {
    "blue":   [(100, 80, 50), (130, 255, 255), 0],
    "red":    [(0, 80, 50), (10, 255, 255), 1],
    "orange": [(10, 80, 50), (25, 255, 255), 2],
    "yellow": [(20, 80, 50), (35, 255, 255), 3],
    "purple": [(130, 80, 50), (155, 255, 255), 4],
    "brown":  [(10, 60, 20), (20, 255, 200), 5],
}

# ==== OUTPUT FOLDERS ====
img_dir = f"live_autolabel/images/{color_name}"
lbl_dir = f"live_autolabel/labels/{color_name}"
os.makedirs(img_dir, exist_ok=True)
os.makedirs(lbl_dir, exist_ok=True)

def get_jetbot_frame(jetbot_ip):
    response = requests.get(f'http://{jetbot_ip}:8080/camera', timeout=2)
    image = Image.open(io.BytesIO(response.content))
    return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

def is_square(cnt, epsilon=0.06):
    peri = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, epsilon * peri, True)
    if len(approx) == 4 and cv2.isContourConvex(approx):
        x, y, w, h = cv2.boundingRect(approx)
        ar = float(w) / h
        return 0.8 < ar < 1.2
    return False

print(f"\n=== Ready? Hold up '{color_name.upper()}' paper and press Enter to start! ===")
input()

lower, upper, class_id = color_hsv_ranges[color_name]
min_square_area = 500
count = 0

try:
    while count < desired_images:
        frame = get_jetbot_frame(jetbot_ip)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, np.array(lower, dtype=np.uint8), np.array(upper, dtype=np.uint8))
        kernel = np.ones((9,9), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        valid_boxes = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area < min_square_area:
                continue
            if not is_square(cnt):
                continue
            x, y, w, h = cv2.boundingRect(cnt)
            img_h, img_w = frame.shape[:2]
            x_center = (x + w/2) / img_w
            y_center = (y + h/2) / img_h
            w_norm = w / img_w
            h_norm = h / img_h
            valid_boxes.append((x, y, w, h, x_center, y_center, w_norm, h_norm))

        show_frame = frame.copy()
        if len(valid_boxes) == 1:
            # Only save if exactly one square of the color is detected!
            x, y, w, h, x_center, y_center, w_norm, h_norm = valid_boxes[0]
            # Draw box for preview
            cv2.rectangle(show_frame, (x, y), (x + w, y + h), (0,255,0), 3)
            cv2.putText(show_frame, color_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
            # Save
            timestamp = datetime.now().strftime("%Y%m%d-%H%M%S-%f")
            img_name = f"{color_name}_{timestamp}.jpg"
            img_path = os.path.join(img_dir, img_name)
            lbl_path = os.path.join(lbl_dir, img_name.replace('.jpg', '.txt'))
            cv2.imwrite(img_path, frame)
            with open(lbl_path, "w") as f:
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")
            count += 1
            msg = f"[{count}/{desired_images}] Saved: {img_name}"
        elif len(valid_boxes) > 1:
            for x, y, w, h, *_ in valid_boxes:
                cv2.rectangle(show_frame, (x, y), (x + w, y + h), (0,0,255), 2)
            msg = f"Too many squares detected!"
        else:
            msg = f"No square found!"

        # Show in notebook
        clear_output(wait=True)
        plt.imshow(cv2.cvtColor(show_frame, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(msg)
        display(plt.gcf())
        plt.close()
        time.sleep(1/fps)
except KeyboardInterrupt:
    print("\nStopped early.")

print(f"\nDone! {count} images with label saved in '{img_dir}' and '{lbl_dir}'.")



=== Ready? Hold up 'YELLOW' paper and press Enter to start! ===


In [1]:
import os
import shutil
import random

# Set paths
src_img_dirs = ['live_autolabel/images/blue', 'live_autolabel/images/yellow']
src_lbl_dirs = ['live_autolabel/labels/blue', 'live_autolabel/labels/yellow']
dst_base = 'yolo_dataset'
os.makedirs(dst_base, exist_ok=True)
img_train_dir = os.path.join(dst_base, 'images/train')
img_val_dir = os.path.join(dst_base, 'images/val')
lbl_train_dir = os.path.join(dst_base, 'labels/train')
lbl_val_dir = os.path.join(dst_base, 'labels/val')
for d in [img_train_dir, img_val_dir, lbl_train_dir, lbl_val_dir]:
    os.makedirs(d, exist_ok=True)

# Collect and shuffle
all_img_files = []
for d in src_img_dirs:
    all_img_files += [os.path.join(d, f) for f in os.listdir(d) if f.endswith('.jpg')]

random.shuffle(all_img_files)
split_idx = int(0.8 * len(all_img_files))
train_imgs = all_img_files[:split_idx]
val_imgs = all_img_files[split_idx:]

def move_files(img_list, img_dst, lbl_dst):
    for img_path in img_list:
        base = os.path.basename(img_path)
        # Determine label source (blue or yellow)
        if "blue" in img_path:
            lbl_path = os.path.join('live_autolabel/labels/blue', base.replace('.jpg', '.txt'))
        else:
            lbl_path = os.path.join('live_autolabel/labels/yellow', base.replace('.jpg', '.txt'))
        shutil.copy2(img_path, os.path.join(img_dst, base))
        shutil.copy2(lbl_path, os.path.join(lbl_dst, base.replace('.jpg', '.txt')))

move_files(train_imgs, img_train_dir, lbl_train_dir)
move_files(val_imgs, img_val_dir, lbl_val_dir)
print("Dataset prepared for YOLOv8!")


Dataset prepared for YOLOv8!


In [11]:
import os

# Force YOLOv8 to use CPU only (no CUDA!)
os.environ["CUDA_VISIBLE_DEVICES"] = ""

from ultralytics import YOLO

# Model training on CPU
model = YOLO('yolov8n.yaml')  # Nano model (fast for small data)
model.train(
    data='yolo_dataset/data.yaml',
    epochs=100,
    imgsz=224,
    device='cpu'   # Explicitly set CPU device
)


Ultralytics 8.3.129  Python-3.8.12 torch-1.13.1+cu117 CPU (AMD Ryzen 7 4800HS with Radeon Graphics)
WARNING Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train9, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_m

train: Scanning C:\Users\fredr\BTH\Robotics\yolo_dataset\labels\train.cache... 60 images, 0 backgrounds, 19 corrupt: 100%|██████████| 60/60 [00:00<?, ?it/s]

train: C:\Users\fredr\BTH\Robotics\yolo_dataset\images\train\yellow_20250521-111331-843235.jpg: ignoring corrupt image/label: Label class 3 exceeds dataset class count 2. Possible class labels are 0-1
train: C:\Users\fredr\BTH\Robotics\yolo_dataset\images\train\yellow_20250521-111341-163937.jpg: ignoring corrupt image/label: Label class 3 exceeds dataset class count 2. Possible class labels are 0-1
train: C:\Users\fredr\BTH\Robotics\yolo_dataset\images\train\yellow_20250521-111341-458466.jpg: ignoring corrupt image/label: Label class 3 exceeds dataset class count 2. Possible class labels are 0-1
train: C:\Users\fredr\BTH\Robotics\yolo_dataset\images\train\yellow_20250521-111344-980589.jpg: ignoring corrupt image/label: Label class 3 exceeds dataset class count 2. Possible class labels are 0-1
train: C:\Users\fredr\BTH\Robotics\yolo_dataset\images\train\yellow_20250521-111351-399526.jpg: ignoring corrupt image/label: Label class 3 exceeds dataset class count 2. Possible class labels are


val: Scanning C:\Users\fredr\BTH\Robotics\yolo_dataset\labels\val.cache... 16 images, 0 backgrounds, 3 corrupt: 100%|██████████| 16/16 [00:00<?, ?it/s]

val: C:\Users\fredr\BTH\Robotics\yolo_dataset\images\val\yellow_20250521-111356-646527.jpg: ignoring corrupt image/label: Label class 3 exceeds dataset class count 2. Possible class labels are 0-1
val: C:\Users\fredr\BTH\Robotics\yolo_dataset\images\val\yellow_20250521-111358-113429.jpg: ignoring corrupt image/label: Label class 3 exceeds dataset class count 2. Possible class labels are 0-1
val: C:\Users\fredr\BTH\Robotics\yolo_dataset\images\val\yellow_20250521-111424-258156.jpg: ignoring corrupt image/label: Label class 3 exceeds dataset class count 2. Possible class labels are 0-1
Plotting labels to runs\detect\train9\labels.jpg... 


...

                   all         13         13      0.925      0.846      0.958      0.636



100 epochs completed in 0.082 hours.
Optimizer stripped from runs\detect\train9\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train9\weights\best.pt, 6.2MB

Validating runs\detect\train9\weights\best.pt...
Ultralytics 8.3.129  Python-3.8.12 torch-1.13.1+cu117 CPU (AMD Ryzen 7 4800HS with Radeon Graphics)
YOLOv8n summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.24it/s]


                   all         13         13       0.82      0.923       0.97      0.654
                  blue         13         13       0.82      0.923       0.97      0.654
Speed: 0.2ms preprocess, 18.1ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\train9


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001C4A885AE50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [ ]:
import torch
torch.cuda.is_available = lambda : False  # <--- Prevent CUDA NMS error

import cv2
import numpy as np
import requests
from PIL import Image
import io

from ultralytics import YOLO

jetbot_ip = '194.47.156.140'

KNOWN_WIDTH = 3.5  # cm (object width for distance calculation)
FOCAL_LENGTH = 500  # Camera calibration

# Load YOLOv8 model
model = YOLO("runs/detect/train9/weights/best.pt")

def get_jetbot_frame(jetbot_ip):
    # Get image from Jetbot web server
    response = requests.get(f'http://{jetbot_ip}:8080/camera')
    image = Image.open(io.BytesIO(response.content))
    frame = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    return frame

while True:
    try:
        frame = get_jetbot_frame(jetbot_ip)
        results = model(frame)
        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()  # xyxy
            confs = result.boxes.conf.cpu().numpy()
            clss = result.boxes.cls.cpu().numpy()

            for (box, conf, cls_id) in zip(boxes, confs, clss):
                x1, y1, x2, y2 = map(int, box)
                w = x2 - x1
                h = y2 - y1
                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
                # Class/confidence label
                label = f"Conf: {conf:.2f} Class: {int(cls_id)}"
                cv2.putText(frame, label, (x1, max(0, y1-10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                # Distance below box
                distance_cm = (KNOWN_WIDTH * FOCAL_LENGTH) / max(w, 1)
                cv2.putText(frame, f"Dist: {distance_cm:.1f}cm", (x1, y2+25),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)

        # Show live video
        cv2.imshow("Jetbot YOLOv8 Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    except Exception as e:
        print(f"Error: {e}")

cv2.destroyAllWindows()



0: 224x224 (no detections), 97.6ms
Speed: 2.0ms preprocess, 97.6ms inference, 0.7ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 82.8ms
Speed: 1.9ms preprocess, 82.8ms inference, 0.7ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 59.2ms
Speed: 1.2ms preprocess, 59.2ms inference, 0.6ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 51.7ms
Speed: 1.2ms preprocess, 51.7ms inference, 0.9ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 55.3ms
Speed: 1.2ms preprocess, 55.3ms inference, 0.7ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 47.3ms
Speed: 1.4ms preprocess, 47.3ms inference, 0.6ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 49.8ms
Speed: 1.3ms preprocess, 49.8ms inference, 0.6ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 49.9ms
Speed: 1.7ms preprocess, 49.9ms i

KeyboardInterrupt: 

: 